In [ ]:
import numpy as np
import pandas as pd


In [7]:
def train_test(X, p=0.8):
    np.random.shuffle(X)
    cant = np.ceil(len(X) * p)
    return X[:int(cant)], X[int(cant):]

In [ ]:
data = pd.read_csv('data.csv')
train, test = train_test(data.to_numpy())
X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

0.0    289
1.0    111
dtype: int64

Ejercicio 2

2.1.

In [23]:
from sklearn.tree import DecisionTreeClassifier
arbol = DecisionTreeClassifier(max_depth=3)

2.2.

In [24]:
# usamos cross_validate porque permite obtener el error en train
from sklearn.model_selection import cross_validate

results = pd.DataFrame(np.nan, index=range(6), columns=['Permutation', 'Accuracy Train', 'Accuracy Val', 'AUCROC Train', 'AUCROC Test', 'AUPRC Train', 'AUPRC Test'])

#cross val con accuracy
k = 5
accuracy_scores = cross_validate(arbol, X_train, y_train, cv=k, return_train_score=True)
accuracy_train_score, accuracy_test_score = accuracy_scores['train_score'], accuracy_scores['test_score']

# cross val con AUCROC
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score
aucroc_scorer = make_scorer(roc_auc_score)
aucroc_scores = cross_validate(arbol, X_train, y_train, cv=k, scoring=aucroc_scorer, return_train_score=True)
aucroc_train_score, aucroc_test_score = aucroc_scores['train_score'], aucroc_scores['test_score']

# cross val con AUPRC
auprc_scorer = make_scorer(average_precision_score)
auprc_scores = cross_validate(arbol, X_train, y_train, cv=k, scoring=auprc_scorer, return_train_score=True)
auprc_train_score, auprc_test_score = auprc_scores['train_score'], auprc_scores['test_score']

for i in range(0, 5):
    results.loc[i] = np.array([int(i+1), accuracy_train_score[i], accuracy_test_score[i], aucroc_train_score[i], aucroc_test_score[i], auprc_train_score[i], auprc_test_score[i]])

results.loc[5] = ["PROMEDIOS", accuracy_train_score.mean(), accuracy_test_score.mean(), aucroc_train_score.mean(), aucroc_test_score.mean(), auprc_train_score.mean(), auprc_test_score.mean()]
results
#AGREGAR SCORE GLOBAL ¿¿

,Permutation,Accuracy Train,Accuracy Val,AUCROC Train,AUCROC Test,AUPRC Train,AUPRC Test
0,1.0,0.803125,0.6625,0.787765,0.639881,0.571992,0.388441
1,2.0,0.825000,0.6750,0.764569,0.586310,0.594130,0.356863
2,3.0,0.843750,0.6125,0.779095,0.509091,0.628954,0.312500
3,4.0,0.796875,0.6625,0.719664,0.558182,0.531525,0.355667
4,5.0,0.781250,0.6500,0.651407,0.516364,0.497790,0.320682
5,PROMEDIOS,0.810000,0.6525,0.740500,0.561965,0.564878,0.346830


2.3.

In [29]:
criterios = ["gini", "entropy"]
alturas = [3, 5, None]

grid = pd.DataFrame(np.nan, index=range(6), columns = ['Altura máxima', 'Criterio de corte', 'Accuracy Train', 'Accuracy Val'])

i = 0
for criterio in criterios:
    for h in alturas:
        res = cross_validate(DecisionTreeClassifier(max_depth=h, criterion=criterio), X_train, y_train, cv=k, return_train_score=True)
        if h:
            grid.loc[i] = [h, criterio, res["train_score"].mean(), res["test_score"].mean()]
        else:
            grid.loc[i] = ['INF', criterio, res["train_score"].mean(), res["test_score"].mean()]
        i+=1
        
grid

,Altura máxima,Criterio de corte,Accuracy Train,Accuracy Val
0,3.0,gini,0.810000,0.6600
1,5.0,gini,0.927500,0.6225
2,INF,gini,1.000000,0.6350
3,3,entropy,0.766250,0.6125
4,5,entropy,0.873125,0.6450
5,INF,entropy,1.000000,0.6225


2.4.

Observando la primera tabla, vemos que existe una diferencia importante entre los resultados para distintas métricas. En particular, al usar Cross Validation con accuracy se obtienen mejores resultados que con AUCROC y, sobre todo, al comparar accuracy con AUPRC. Esta diferencia en rendimiento podría deberse a un desbalanceo entre clases, especialmente si consideramos el contexto de nuestro problema: debería ser más probable que alguien no esté enfermo que que sí. Efectivamente, podemos comprobar que existe un desbalanceo entre clases; particularmente, hay un preponderancia de la clase negativa por sobre la positiva.


In [33]:
pd.Series(y_train).value_counts()

0.0    289
1.0    111
dtype: int64

 Al dejar altura infinita, overfittea (un poco). No hay casi diferencia entre Gini y entropia

(320, 200) (320,)
(320, 200) (320,)
(320, 200) (320,)
(320, 200) (320,)
(320, 200) (320,)
Cross-validation scores (custom): [[0.66454545 0.54761905]
 [0.79415954 0.5       ]
 [0.72213895 0.49820789]
 [0.81317702 0.52909091]
 [0.79041488 0.61942175]]
Cross-validation mean score (custom): [0.75688717 0.53886792]
